In [1]:
import numpy as np
import pandas as pd
import os
import ROOT

Welcome to JupyROOT 6.16/00


In [2]:
def neuland_calorimetric_data(distance, doubleplane, energy, erel, nmax):
    filepattern = "output/%dm_%ddp_%dAMeV_%dkeV_%dn.%s.root"
    ncutfile = filepattern % (distance, doubleplane, energy, erel, nmax, "ncut")

    file = ROOT.TFile.Open(ncutfile)
    para = file.Get("R3BNeulandNeutron2DPar")

    data = []
    num_ex = 0
    for neutron in range(1, nmax + 1):
        digifile = filepattern % (distance, doubleplane, energy, erel, neutron, "digi")
        tfile = ROOT.TFile.Open(digifile)
        ttree = tfile.Get("evt")
        for event in ttree:
            # nPN: Number of incoming primary neutrons
            nIn = neutron
            # nPP: Number of primary neutrons with an energy deposition in NeuLAND
            nPP = event.NeulandPrimaryPoints.GetEntries()
            # nPH: Number of hits that correspond to a energy deposition of a primary neutron
            nPH = event.NeulandPrimaryHits.GetEntries()
            # nHits: Number of hits
            num_hits = event.NeulandHits.GetEntries()
            # nClus: Number of clusters
            num_clusters = event.NeulandClusters.GetEntries()
            # Edep: Total deposited (detected) energy
            try:
                edep = round(sum([hit.GetE() for hit in event.NeulandHits]))
            except:
                num_ex += 1
                edep = 0

            # Use calibrated calorimetric method (R3BNeulandNeutron2DPar) from the parameter file
            # to predict the multiplicity
            mult = para.GetNeutronMultiplicity(edep, num_clusters)

            data.append([nIn, nPP, nPH, num_hits, num_clusters, edep, mult])
    datapd = pd.DataFrame(data)
    datapd.columns = ["nIn", "nPP", "nPH", "num_hits", "num_cluster", "edep", "mult"]
    return datapd

In [3]:
distance = 15
doubleplane = 30
erel = 500
nmax = 5

data = {
    energy: neuland_calorimetric_data(distance, doubleplane, energy, erel, nmax)
    for energy in [200, 600, 1000]
}

pd.options.display.max_rows = 20
display(data[600])

,nIn,nPP,nPH,num_hits,num_cluster,edep,mult
0,1,1,1,8,8,150,1
1,1,1,1,11,5,314,1
2,1,1,1,30,5,420,2
3,1,1,1,13,3,245,1
4,1,1,1,2,1,115,1
...,...,...,...,...,...,...,...
49995,5,5,5,72,20,1430,4
49996,5,5,5,57,18,1088,4
49997,5,5,5,67,19,1529,5
49998,5,5,5,59,30,1115,4


Warning in <TFile::Init>: file output/15m_30dp_200AMeV_500keV_5n.ncut.root probably not closed, trying to recover
Info in <TFile::Recover>: output/15m_30dp_200AMeV_500keV_5n.ncut.root, recovered key R3BNeulandNeutron2DPar:R3BNeulandNeutron2DPar at address 282
Info in <TFile::Recover>: output/15m_30dp_200AMeV_500keV_5n.ncut.root, recovered key FairRtdbRun:1 at address 917
Info in <TFile::Recover>: output/15m_30dp_200AMeV_500keV_5n.ncut.root, recovered key TH2D:hEC0 at address 1104
Info in <TFile::Recover>: output/15m_30dp_200AMeV_500keV_5n.ncut.root, recovered key TH2D:hEC1 at address 2047
Info in <TFile::Recover>: output/15m_30dp_200AMeV_500keV_5n.ncut.root, recovered key TH2D:hEC2 at address 3492
Info in <TFile::Recover>: output/15m_30dp_200AMeV_500keV_5n.ncut.root, recovered key TH2D:hEC3 at address 5396
Info in <TFile::Recover>: output/15m_30dp_200AMeV_500keV_5n.ncut.root, recovered key TH2D:hEC4 at address 7683
Info in <TFile::Recover>: output/15m_30dp_200AMeV_500keV_5n.ncut.root, 

In [4]:
from sklearn.metrics import (
    balanced_accuracy_score,
    confusion_matrix,
    plot_confusion_matrix,
)

np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

out = []
for energy, df in data.items():
    print(energy)
    y_true = df[["nIn"]].values.ravel()
    y_pred = df[["mult"]].values.ravel()

    bac = balanced_accuracy_score(y_true, y_pred)
    print(bac)
    # cm = confusion_matrix(y_true, y_pred, labels=range(0, 6))
    # print(np.swapaxes(cm, 0, 1))
    cmrel = confusion_matrix(y_true, y_pred, labels=range(0, 6), normalize="true")
    cmrel = np.swapaxes(cmrel, 0, 1)
    print(cmrel)
    out.append([energy, bac, cmrel])

200
0.66708
[[0.    0.101 0.005 0.    0.    0.   ]
 [0.    0.853 0.176 0.02  0.002 0.   ]
 [0.    0.046 0.715 0.25  0.037 0.004]
 [0.    0.    0.103 0.597 0.282 0.061]
 [0.    0.    0.    0.131 0.536 0.301]
 [0.    0.    0.    0.001 0.143 0.634]]
600
0.6706799999999999
[[0.    0.041 0.002 0.    0.    0.   ]
 [0.    0.835 0.122 0.008 0.001 0.   ]
 [0.    0.124 0.762 0.224 0.033 0.004]
 [0.    0.    0.115 0.678 0.327 0.074]
 [0.    0.    0.    0.09  0.57  0.413]
 [0.    0.    0.    0.    0.069 0.509]]
1000
0.69374
[[0.    0.026 0.001 0.    0.    0.   ]
 [0.    0.824 0.092 0.005 0.    0.   ]
 [0.    0.15  0.782 0.188 0.022 0.003]
 [0.    0.    0.126 0.714 0.302 0.06 ]
 [0.    0.    0.    0.093 0.615 0.404]
 [0.    0.    0.    0.    0.061 0.534]]


/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [5]:
def to_tabular(energy, bac, cm):
    text = """\\resizebox{0.32\\textwidth}{!}{
\\begin{tabular}{cc|S[table-format=2]S[table-format=2]S[table-format=2]S[table-format=2]S[table-format=2]}
\\toprule
\\multicolumn{2}{c|}{""" + str(energy) + """} & \\multicolumn{5}{c}{generated} \\\\
\\multicolumn{2}{c|}{""" + str("MeV") + """} & 1 & 2 & 3 & 4 & 5 \\\\
\\midrule
\\multirow{6}{*}{\\rotatebox[origin=c]{90}{detected}}
"""
    for i, a in enumerate(cm):
        text += " & " +  str(i)
        # text += " & ".join([ for x in a[1:]])
        for j, x in enumerate(a[1:]):
            if j == i - 1: 
                text += " & \\textbf{" + str(int(round(x*100))) + "}"
            else:
                text += " & " + str(int(round(x*100)))
        text += " \\\\\n"
    text += """\\bottomrule
\\end{tabular}}"""
    return text

In [8]:
def to_full_latex(data):
    text = """\\documentclass{scrartcl}
\\usepackage{booktabs}
\\usepackage{siunitx}
\\usepackage{multirow}
\\usepackage{graphicx}
\\begin{document}
\\begin{table}
\\captionabove[Neutron separation matrices for multiplicities of 1 to 5 neutrons]{
Neutron separation matrices for multiplicities of 1 to 5 neutrons.
Columns display the neutron multiplicity simulated, rows the neutron multiplicity derived from the neutron tracking algorithm.
Values are given in percent.
Neutrons were simulated with 200 (left), 600 (center) and 1000 MeV (right matrix).
NeuLAND was located at a distance of 15 m to the target.
Neutrons were generated with a relative energy of 500 keV with respect to a medium heavy projectile fragment.
\\label{tab:neutron-separation-matrices}}
"""
    for o in out:
        text += to_tabular(*o) + "\n"
    text += """\\end{table}
\\end{document}
"""
    return text

In [9]:
latex = to_full_latex(out)
print(latex)
with open("paper/mult.tex", "w") as f:
    f.write(latex)
import subprocess

subprocess.call(["lualatex", "mult.tex"], cwd="paper")

\documentclass{scrartcl}
\usepackage{booktabs}
\usepackage{siunitx}
\usepackage{multirow}
\usepackage{graphicx}
\begin{document}
\begin{table}
\captionabove[Neutron separation matrices for multiplicities of 1 to 5 neutrons]{
Neutron separation matrices for multiplicities of 1 to 5 neutrons.
Columns display the neutron multiplicity simulated, rows the neutron multiplicity derived from the neutron tracking algorithm.
Values are given in percent.
Neutrons were simulated with 200 (left), 600 (center) and 1000 MeV (right matrix).
NeuLAND was located at a distance of 15 m to the target.
Neutrons were generated with a relative energy of 500 keV with respect to a medium heavy projectile fragment.
\label{tab:neutron-separation-matrices}}
\resizebox{0.32\textwidth}{!}{
\begin{tabular}{cc|S[table-format=2]S[table-format=2]S[table-format=2]S[table-format=2]S[table-format=2]}
\toprule
\multicolumn{2}{c|}{200} & \multicolumn{5}{c}{generated} \\
\multicolumn{2}{c|}{MeV} & 1 & 2 & 3 & 4 & 5 \\
\midrul

0

This is LuaTeX, Version 1.0.4 (TeX Live 2017) 
 restricted system commands enabled.
(./mult.tex
LaTeX2e <2017-04-15>
(using write cache: /home/jmayer/.texlive2017/texmf-var/luatex-cache/generic)(u
sing read cache: /usr/local/texlive/2017/texmf-var/luatex-cache/generic /home/j
mayer/.texlive2017/texmf-var/luatex-cache/generic)
luaotfload | main : initialization completed in 0.115 seconds
Babel <3.12> and hyphenation patterns for 1 language(s) loaded.
(/usr/local/texlive/2017/texmf-dist/tex/latex/koma-script/scrartcl.cls
Document Class: scrartcl 2017/04/13 v3.23 KOMA-Script document class (article)
(/usr/local/texlive/2017/texmf-dist/tex/latex/koma-script/scrkbase.sty
(/usr/local/texlive/2017/texmf-dist/tex/latex/koma-script/scrbase.sty
(/usr/local/texlive/2017/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/local/texlive/2017/texmf-dist/tex/latex/koma-script/scrlfile.sty
Package scrlfile, 2017/04/13 v3.23 KOMA-Script package (loading files)
                  Copyright (C) Markus Kohm

)